In [14]:
from decouple import config
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from openai import OpenAI

OPEN_AI_API_KEY = config('OPEN_AI_KEY')
UPSTASH_VECTOR_URL = config('UPSTASH_VECTOR_ENDPOINT')
UPSTASH_VECTOR_TOKEN = config('UPSTASH_VECTOR_TOKEN')

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import UpstashVectorStore

In [3]:
embeddings = OpenAIEmbeddings(
    model='text-embedding-3-small',
    openai_api_key=OPEN_AI_API_KEY
    )

In [4]:
store = UpstashVectorStore(
    embedding=embeddings,
    index_url=UPSTASH_VECTOR_URL,
    index_token=UPSTASH_VECTOR_TOKEN
)

In [6]:
retriever = store.as_retriever(
    search_type='similarity',
    search_kwargs={'k':2}
)

In [9]:
LLM_CONFIG = {
    "api_key": OPEN_AI_API_KEY,
    "model": 'gpt-4o-mini'
}

In [16]:
model = ChatOpenAI(**LLM_CONFIG)
parser = StrOutputParser()

In [13]:
message = """
Answer this question using the provided context only.
{question}

Context:
{context}
"""

In [18]:
prompt_template = ChatPromptTemplate.from_messages([
        ("human", message),
    ])

In [19]:
chain = {"context":retriever, "question":RunnablePassthrough()} | prompt_template | model | parser

In [22]:
chain.invoke("What is Cavite known for?")

'Cavite is known for its historical significance, particularly as the site of the "First Cry of Cavite," which was an uprising led by Filipino revolutionaries in 1896. Additionally, Cavite is recognized for its industrial areas, such as the Cavite Export Processing Zone (CEPZ), which hosts numerous factories.'